<a href="https://colab.research.google.com/github/Haebuk/kuggle/blob/main/gamestop_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install wandb -qqq

     |████████████████████████████████| 1.6 MB 8.4 MB/s 
     |████████████████████████████████| 97 kB 8.2 MB/s 
     |████████████████████████████████| 170 kB 42.7 MB/s 
     |████████████████████████████████| 133 kB 56.7 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 


In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
def train():
    import wandb 
    from wandb.keras import WandbCallback
    import tensorflow as tf
    import pandas as pd
    import numpy as np
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout
    from sklearn.model_selection import train_test_split

    PATH = '/content/drive/MyDrive/input/'
    df = pd.read_csv(PATH + 'GME_scaled.csv')

    default_configs = {
        'learning_rate': 0.001,
        'dropout_rate1': 0.2,
        'dropout_rate2': 0.2,
        'dropout_rate3': 0.2,
        'hidden1': 50,
        'hidden2': 50,
        'hidden3': 50,
        'time_step': 60,
    }
    wandb.init(project='gamestop_prediction', config=default_configs, magic=True)
    config = wandb.config

    X = df.drop('date', axis=1).values
    y = df['open_price'].values
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

    time_step = config.time_step
    X_train, y_train, X_test, y_test = [], [], [], []
    for i in range(time_step,len(X_tr)): # train
        X_train.append(X_tr[i-time_step:i,:])
        y_train.append(y_tr[i])
    for i in range(time_step, len(X_te)): # test
        X_test.append(X_te[i-time_step:i,:])
        y_test.append(y_te[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)
    # LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
    X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
    X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

    model = tf.keras.Sequential([
    LSTM(config.hidden1, return_sequences=True, input_shape=(config.time_step, 2)),
    Dropout(config.dropout_rate1),
    LSTM(config.hidden2, return_sequences=True),
    Dropout(config.dropout_rate2),
    LSTM(config.hidden3),
    Dropout(config.dropout_rate3),
    Dense(1),
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    model.compile(optimizer=opt,
                loss='mse',
                metrics=['mse'])
    
    model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=200, callbacks=[WandbCallback(), early_stopping])

In [14]:
with open('train.py', 'w') as f:
    f.write("""
import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

PATH = '/content/drive/MyDrive/input/'
df = pd.read_csv(PATH + 'GME_scaled.csv')

default_configs = {
    'learning_rate': 0.001,
    'dropout_rate1': 0.2,
    'dropout_rate2': 0.2,
    'dropout_rate3': 0.2,
    'hidden1': 50,
    'hidden2': 50,
    'hidden3': 50,
    'time_step': 60,
}
wandb.init(project='gamestop_prediction', config=default_configs, magic=True)
config = wandb.config

X = df.drop('date', axis=1).values
y = df['open_price'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

time_step = config.time_step
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(time_step,len(X_tr)): # train
    X_train.append(X_tr[i-time_step:i,:])
    y_train.append(y_tr[i])
for i in range(time_step, len(X_te)): # test
    X_test.append(X_te[i-time_step:i,:])
    y_test.append(y_te[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
# LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

model = tf.keras.Sequential([
LSTM(config.hidden1, return_sequences=True, input_shape=(config.time_step, 2)),
Dropout(config.dropout_rate1),
LSTM(config.hidden2, return_sequences=True),
Dropout(config.dropout_rate2),
LSTM(config.hidden3),
Dropout(config.dropout_rate3),
Dense(1),
])

opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(optimizer=opt,
            loss='mse',
            metrics=['mse'])

model.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200, callbacks=[WandbCallback(), early_stopping])
    """)

In [15]:
!head train.py


import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split



In [17]:
!wandb agent kade/gamestop_prediction/47ty4gib

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
	dropout_rate1: 0.16478566479409007
	dropout_rate2: 0.056896766413966106
	dropout_rate3: 0.38967783541929096
	hidden1: 81
	hidden2: 89
	hidden3: 100
	learning_rate: 0.007916615730729864
	time_step: 25
2021-08-19 14:15:46,348 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --dropout_rate1=0.16478566479409007 --dropout_rate2=0.056896766413966106 --dropout_rate3=0.38967783541929096 --hidden1=81 --hidden2=89 --hidden3=100 --learning_rate=0.007916615730729864 --time_step=25
wandb: Currently logged in as: kade (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: Tracking run with wandb version 0.12.0
wandb: Syncing run earnest-sweep-583
wandb: ⭐️ View project at https://wandb.ai/kade/gamestop_prediction
wandb: 🧹 View sweep at https://wandb.ai/kade/gamestop_prediction/sweeps/47ty4gib
wandb: 🚀 View run at https://wandb.ai/kade/gamestop_prediction/runs/b69srk